#Loding libraries,  Modelue etc

In [0]:
from pyspark.sql.types import StructType,StructField, StringType,IntegerType
from pyspark.sql.functions import to_date, col, split, explode, trim,concat_ws, round,year
import pandas as pd

#1: Create raw tables for each source dataset </b>
Data Ingestion

Creating TDD for order table to check <br/>
 -> File data Schema <br/>
 -> File is not empty <br/>
<br/>
Repeat for customer and product data with appropriate format and options

In [0]:
def test_order_schema():
  assert order_df.schema.fields[0].name == "Customer ID"  # Replace with actual column names based on data
  assert order_df.schema.fields[0].dataType == StringType()  # Replace with data types based on data

  assert product_df.schema.fields[0].name == "Product ID"  # Replace with actual column names based on data
  assert product_df.schema.fields[0].dataType == StringType()  # Replace with data types based on data

  assert customer_df.schema.fields[0].name == "Customer ID"  # Replace with actual column names based on data
  assert customer_df.schema.fields[0].dataType == StringType()  # Replace with data types based on data


# Replace "/path/to/Order.json" with the actual location of your data
def read_data_with_schema_check(filepath, format="json", multiLine=True,header=True):
  try:
    df = spark.read.format(format).option("multiLine", multiLine).option("header",header).load(filepath)
   # df.show(truncate=False)  # Show all data to identify potential issues
    return df
  except FileNotFoundError:
    raise Exception("File not found: " + filepath)



#order raw data
try:
  order_df = read_data_with_schema_check("dbfs:/FileStore/shared_uploads/aman.nagariya@hm.com/Order.json")
  test_order_schema()
  print("Order schema test passed!")
except Exception as e:
  print("Error:", e)


#Product raw data
try:
  product_df = read_data_with_schema_check("dbfs:/FileStore/shared_uploads/aman.nagariya@hm.com/Product.csv", format="csv", header=True)
  test_order_schema()
  print("Product schema test passed!")
except Exception as e:
  print("Error:", e)


#Customer Raw data
try:
  #reading excle file into Datafrme
  #read excle file into Pandas dataframe then convert it into spark DF
  wb = pd.read_excel('/dbfs/FileStore/shared_uploads/aman.nagariya@hm.com/Customer.xlsx')
  # Specifing the schema for the phone column as StringType as its creating conflict due to sting and long both the datatype
  schema = StructType([
    StructField("Customer ID", StringType(), nullable=True),
    StructField("Customer Name", StringType(), nullable=True),
    StructField("email", StringType(), nullable=True),
    StructField("phone", StringType(), nullable=True),
    StructField("address", StringType(), nullable=True),
    StructField("Segment", StringType(), nullable=True),
    StructField("Country", StringType(), nullable=True),
    StructField("City", StringType(), nullable=True),
    StructField("State", StringType(), nullable=True),
    StructField("Postal Code", StringType(), nullable=True),
    StructField("Region", StringType(), nullable=True)
    ])
  customer_df=spark.createDataFrame(wb,schema)
  
  test_order_schema()
  print("Customer schema test passed!")
except Exception as e:
  print("Error:", e)

Order schema test passed!
Product schema test passed!
Customer schema test passed!


data undertanding and cleaning/pre-processing

In [0]:
print("custoemr_df : row :", customer_df.count() )
print("order_df : row :", order_df.count() )
print("product_df : row :", product_df.count() )

custoemr_df : row : 793
order_df : row : 9994
product_df : row : 1851

In [0]:
#droping duplicate if any
customer_df=customer_df.dropDuplicates()
order_df=order_df.dropDuplicates()
product_df=product_df.dropDuplicates()

In [0]:
print("custoemr_df : row :", customer_df.count() )
print("order_df : row :", order_df.count() )
print("product_df : row :", product_df.count() )

custoemr_df : row : 793
order_df : row : 9994
product_df : row : 1851

In [0]:
# Convert 'order_date' column to date type
order_df = order_df.withColumn('Order Date', to_date(order_df['Order Date'], 'd/M/yyyy'))
order_df = order_df.withColumn('Ship Date', to_date(order_df['Ship Date'], 'd/M/yyyy'))
# Convert 'price per product' column to double
product_df = product_df.withColumn('Price per product', product_df['Price per product'].cast('double'))

lets analyse the data for one customer for better undertanding
<br/>  CustoemrID = SG-20605

In [0]:
#creating VIew 
customer_df.createOrReplaceTempView("customer_t")
order_df.createOrReplaceTempView("order_t")
product_df.createOrReplaceTempView("product_t")

In [0]:
%sql

select *
from customer_t
where `Customer ID` = "SG-20605"

Customer ID,Customer Name,email,phone,address,Segment,Country,City,State,Postal Code,Region
SG-20605,Speros Goranitis,brianjoyce110@gmail.com,3528465094,"02401 Angela Loop Apt. 678 Port John, ME 43448",Consumer,United States,Lafayette,Indiana,47905,Central


In [0]:
%sql

select *
from order_t
where `Customer ID` = "SG-20605"

Customer ID,Discount,Order Date,Order ID,Price,Product ID,Profit,Quantity,Row ID,Ship Date,Ship Mode
SG-20605,0.2,2014-12-19,CA-2014-169446,32.592,OFF-ST-10000419,-7.7406,3,3427,2014-12-25,Standard Class
SG-20605,0.2,2017-09-13,CA-2017-139493,15.92,OFF-AR-10003158,2.786,5,9701,2017-09-19,Standard Class
SG-20605,0.2,2014-12-19,CA-2014-169446,15.552,OFF-PA-10000295,5.4432,3,7431,2014-12-25,Standard Class
SG-20605,0.2,2014-11-03,CA-2014-121573,48.896,OFF-BI-10000666,18.336,2,6048,2014-11-07,Standard Class
SG-20605,0.0,2015-12-06,US-2015-127040,6.48,OFF-PA-10004255,3.1104,1,4456,2015-12-10,Standard Class
SG-20605,0.0,2014-01-23,CA-2014-146997,5.94,OFF-FA-10003467,0.0,3,733,2014-01-27,Standard Class
SG-20605,0.0,2015-12-06,US-2015-127040,41.86,OFF-PA-10001184,20.5114,7,5523,2015-12-10,Standard Class
SG-20605,0.2,2014-11-03,CA-2014-121573,7.856,OFF-BI-10003712,2.8478,2,5520,2014-11-07,Standard Class
SG-20605,0.0,2015-12-06,US-2015-127040,113.92,FUR-FU-10002878,42.1504,4,5772,2015-12-10,Standard Class
SG-20605,0.2,2014-12-19,CA-2014-169446,324,TEC-PH-10002817,36.4473,3,2326,2014-12-25,Standard Class


There are muny customer who placced multiple orders

In [0]:
%sql

select *
from product_t
where `Product ID` = "FUR-FU-10002878"

Product ID,Category,Sub-Category,Product Name,State,Price per product
FUR-FU-10002878,Furniture,Furnishings,"""Seth Thomas 14"""" Day/Date Wall Clock""",New York,28.48


#2: Create an enriched table for customers and products 

no enrichment needed in customer table<br/>
all the fields are having correct representation<br/>
PII data -- we can do the masking, but as this is just a Demo I'm skiping that part<br/>

In product table also there is no enrichment need, but just to showcase how it is done, wrting a simple case of creating the category_hierarchy of products

In [0]:

#TDD test case
def test_enriched_product():
  assert "category_hierarchy" in product_df.columns  # Check for derived column
  assert product_df.filter(product_df.Category.isNull() ).count() == 0  # Handle null values



#Transformation function
def enrich_product(product_df):
  product_df = product_df.withColumn("category_hierarchy", concat_ws(" > ", col("Category"), col("Sub-Category")))
  return product_df


try:
  product_df = enrich_product(product_df)  
  test_enriched_product()
  print("Enriched product table test passed!")
except AssertionError as e:
  print("Enriched product table test failed:", e)

product_df.show(2)

Enriched product table test passed!
+---------------+---------------+------------+--------------------+--------+-----------------+--------------------+
 Product ID| Category|Sub-Category| Product Name| State|Price per product| category_hierarchy|
+---------------+---------------+------------+--------------------+--------+-----------------+--------------------+
OFF-BI-10002824|Office Supplies| Binders|Recycled Easel Ri...|Colorado| 4.25|Office Supplies >...|
FUR-CH-10002961| Furniture| Chairs|Leather Task Chai...|New York| 81.882| Furniture > Chairs|
+---------------+---------------+------------+--------------------+--------+-----------------+--------------------+
only showing top 2 rows

#3:Create an enriched table which has<br/>
->order information 
      <t/><br/>Profit rounded to 2 decimal places<br/>
->Customer name and country<br/>
->Product category and sub category


In [0]:

def round_to_two_decimals(col_name):
  """Rounds a column to two decimal places."""
  return round((col_name), 2)



def test_enriched_table():
  assert "Order ID" in enriched_df.columns  # Check for Order ID
  assert "Customer Name" in enriched_df.columns # Check for customer name
  assert "Country" in enriched_df.columns  # Check for Customer Country
  assert "category" in enriched_df.columns  # Check for Product Category
  assert "sub-category" in enriched_df.columns  # Check for Product Sub Category
  assert "Profit" in enriched_df.columns  # Check for Profit

def enrich_data(order_df, customer_df, product_df):
  
  # Remove duplicates from product_df ( duplicates on ProductID)
  product_df = product_df.dropDuplicates(["Product ID"])  


  # Join orders and customers on Order ID (foreign key relationship)
  enriched_df = order_df.join(customer_df, order_df["Customer ID"]== customer_df["Customer ID"], how='inner')

  # Join enriched table with products on Product ID (assuming foreign key relationship)
  enriched_df = enriched_df.join(product_df, enriched_df["Product ID"] == product_df["Product ID"], how='inner')

  # Select desired columns and round profit  
  enriched_df = enriched_df.select(
      order_df["Order ID"],
      customer_df["Customer Name"],  
      customer_df["Country"],
      product_df["category"],
      product_df["sub-category"],
      round_to_two_decimals(order_df["Profit"]).alias("Profit")  # Use helper function or directly round
  )

  return enriched_df
  

try:
  enriched_df = enrich_data(order_df, customer_df, product_df) 
  test_enriched_table()
  print("Enriched table test passed!")
except AssertionError as e:
  print("Enriched table test failed:", e)


enriched_df.show(5)

Enriched table test passed!
+--------------+------------------+-------------+---------------+------------+-------+
 Order ID| Customer Name| Country| category|sub-category| Profit|
+--------------+------------------+-------------+---------------+------------+-------+
CA-2017-133256| Tracy Hopkins|United States| Technology| Phones| 135.98|
CA-2016-140634| Hunter Lopez|United States|Office Supplies| Envelopes| 5.09|
CA-2016-125087| Thea Hudgings|United States|Office Supplies| Storage| 77.75|
US-2016-114013|Stewart Carmichael|United States| Technology| Phones|-116.98|
CA-2015-119627| Steven Cartwright|United States|Office Supplies| Paper| 12.7|
+--------------+------------------+-------------+---------------+------------+-------+
only showing top 5 rows

In [0]:
%sql
select `product id`, count(`product id`)
from product_t
group by `product id`
having count(`product id`)>1

product id,count(product id)
OFF-PA-10000477,2
FUR-FU-10004270,2
TEC-MA-10001148,2
OFF-BI-10004654,2
FUR-FU-10004017,2
OFF-PA-10001166,2
OFF-PA-10002195,2
OFF-AR-10001149,2
FUR-BO-10002213,2
TEC-PH-10004531,2


There are mutiple entry for same product id in table, and thats the resons you see multiple entry for same order ID <br/>
duplicate rows indicates the product in diffrent states, without having more information about the data, its difficult to make any assumnstion.<br/>
Their are many thing we can do to address these kind of isse:<br/>
-> Remove duplicate from Product table so that each product have only one row i.e more reliable data
-> If combination of product_id and state is going to act as primary key then more specific information in needed like does state represent <br/>the state of product manufacture or its associated with customer state who is buying the product

-> I've added dropduplicate for product table for now, once we have more info we can make more relevent transformation.

#4:Create an aggregate table that shows profit by 
<br/>->Year
<br/>->Product Category
<br/>->Product Sub Category
<br/>->Customer


In [0]:
#adding date, customer id in enrichment df  -- improvment 


def round_to_two_decimals(col_name):
  """Rounds a column to two decimal places."""
  return round((col_name), 2)



def test_enriched_table():
  assert "Order date" in enriched_df.columns  # Check for Order ID
  assert "Order ID" in enriched_df.columns  # Check for Order ID
  assert "Customer Name" in enriched_df.columns # Check for customer name
  assert "Country" in enriched_df.columns  # Check for Customer Country
  assert "category" in enriched_df.columns  # Check for Product Category
  assert "sub-category" in enriched_df.columns  # Check for Product Sub Category
  assert "Profit" in enriched_df.columns  # Check for Profit

def enrich_data(order_df, customer_df, product_df):
  
  # Remove duplicates from product_df ( duplicates on ProductID)
  product_df = product_df.dropDuplicates(["Product ID"])  


  # Join orders and customers on Order ID (foreign key relationship)
  enriched_df = order_df.join(customer_df, order_df["Customer ID"]== customer_df["Customer ID"], how='inner')

  # Join enriched table with products on Product ID (assuming foreign key relationship)
  enriched_df = enriched_df.join(product_df, enriched_df["Product ID"] == product_df["Product ID"], how='inner')

  # Select desired columns and round profit  
  enriched_df_im = enriched_df.select(
      order_df["order date"],
      order_df["Order ID"],
      customer_df["Customer ID"],  
      customer_df["Country"],
      product_df["category"],
      product_df["sub-category"],
      round_to_two_decimals(order_df["Profit"]).alias("Profit")  # Use helper function or directly round
  )

  return enriched_df_im
  

try:
  enriched_df_im = enrich_data(order_df, customer_df, product_df) 
  test_enriched_table()
  print("Enriched table test passed!")
except AssertionError as e:
  print("Enriched table test failed:", e)


enriched_df_im.show(5)
enriched_df_im.createOrReplaceTempView('enriched_im_t')

Enriched table test failed: 
+----------+--------------+-----------+-------------+---------------+------------+-------+
order date| Order ID|Customer ID| Country| category|sub-category| Profit|
+----------+--------------+-----------+-------------+---------------+------------+-------+
2017-06-26|CA-2017-133256| TH-21550|United States| Technology| Phones| 135.98|
2016-10-03|CA-2016-140634| HL-15040|United States|Office Supplies| Envelopes| 5.09|
2016-04-18|CA-2016-125087| TH-21115|United States|Office Supplies| Storage| 77.75|
2016-03-13|US-2016-114013| SC-20770|United States| Technology| Phones|-116.98|
2015-08-17|CA-2015-119627| SC-20725|United States|Office Supplies| Paper| 12.7|
+----------+--------------+-----------+-------------+---------------+------------+-------+
only showing top 5 rows

In [0]:
def test_aggregate_profit(aggregated_df, expected_year=None, expected_category=None, expected_sub_category=None, expected_customer=None):
  """
  Test function for aggregate_profit. Checks presence of columns, data types,
  and optionally verifies specific values for Year, Category, Sub Category,
  Customer ID, and Total Profit.
  """
  assert "Year" in aggregated_df.columns  # Check for Year column
  assert "Product_Category" in aggregated_df.columns  # Check for Product Category
  assert "Product_Sub_Category" in aggregated_df.columns  # Check for Product Sub Category
  assert "Customer" in aggregated_df.columns  # Check for Customer ID
  assert "Total_Profit" in aggregated_df.columns  # Check for aggregated profit
  
  # Optional assertions for specific values (modify as needed)
  if expected_year:
    assert aggregated_df.filter(col("Year") == expected_year).count() > 0  # Check for expected year
  if expected_category:
    assert aggregated_df.filter(col("Product_Category") == expected_category).count() > 0  # Check for expected category
  if expected_sub_category:
    assert aggregated_df.filter(col("Product_Sub_Category") == expected_sub_category).count() > 0  # Check for expected sub category
  if expected_sub_category:
    assert aggregated_df.filter(col("Customer") == expected_customer).count() > 0  # Check for expected sub category
  




def aggregate_profit(enriched_im_t):  
  aggregate_df = spark.sql("""
      SELECT 
          YEAR(`Order Date`) AS Year,
          `Category` AS Product_Category,
          `Sub-Category` AS Product_Sub_Category,
          `Customer ID` AS Customer,
          ROUND(SUM(profit), 2) AS Total_Profit
      FROM 
          enriched_im_t
      GROUP BY 
          YEAR(`Order Date`), 
          `Category`, 
          `Sub-Category`, 
          `Customer ID`
  """)
  return aggregate_df
# Show the aggregate table
#aggregate_table.show()


try:
  aggregate_df = aggregate_profit('enriched_im_t') 
  test_aggregate_profit(aggregate_df)
  print("aggregate table test passed!")
except AssertionError as e:
  print("aggregate table test failed:", e)


aggregate table test passed!

#4:Using SQL output the following aggregates<br/>
->Profit by Year<br/>
->Profit by Year + Product Category
<br/>->Profit by Customer
<br/>->Profit by Customer + Year


In [0]:

def test_profit_by_year(aggregated_df):
  """Test for profit by year."""
  assert "Year" in aggregated_df.columns  # Check for Year column
  assert "Total_Profit" in aggregated_df.columns  # Check for Total Profit

def aggregate_profit_by_year(enriched_im_t):  
  aggregate_df = spark.sql("""
      SELECT 
          YEAR(`Order Date`) AS Year,
          ROUND(SUM(profit), 2) AS Total_Profit
      FROM 
          enriched_im_t
      GROUP BY 
          YEAR(`Order Date`)
  """)
  return aggregate_df
  
def test_profit_by_year_category(aggregated_df):
  """Test for profit by year and product category."""
  assert "Year" in aggregated_df.columns  # Check for Year column
  assert "Product_Category" in aggregated_df.columns  # Check for Product Category
  assert "Total_Profit" in aggregated_df.columns  # Check for Total Profit

def aggregate_profit_by_year_category(enriched_im_t):  
  aggregate_df = spark.sql("""
      SELECT 
          YEAR(`Order Date`) AS Year,
          `Category` AS Product_Category,
          ROUND(SUM(profit), 2) AS Total_Profit
      FROM 
          enriched_im_t
      GROUP BY 
          YEAR(`Order Date`), 
          `Category`
  """)
  return aggregate_df
  
def test_profit_by_customer(aggregated_df):
  """Test for profit by customer."""
  assert "Customer" in aggregated_df.columns  # Check for Customer
  assert "Total_Profit" in aggregated_df.columns  # Check for Total Profit

def aggregate_profit_by_customer(enriched_im_t):  
  aggregate_df = spark.sql("""
      SELECT 
          `Customer ID` AS Customer,
          ROUND(SUM(profit), 2) AS Total_Profit
      FROM 
          enriched_im_t
      GROUP BY 
          `Customer ID`
  """)
  return aggregate_df
  
def test_profit_by_customer_year(aggregated_df):
  """Test for profit by customer and year."""
  assert "Customer" in aggregated_df.columns  # Check for Customer
  assert "Year" in aggregated_df.columns  # Check for Year column
  assert "Total_Profit" in aggregated_df.columns  # Check for Total Profit

def aggregate_profit_customer_by_year(enriched_im_t):  
  aggregate_df = spark.sql("""
      SELECT 
          YEAR(`Order Date`) AS Year,
          `Customer ID` AS Customer,
          ROUND(SUM(profit), 2) AS Total_Profit
      FROM 
          enriched_im_t
      GROUP BY 
          YEAR(`Order Date`), 
          `Customer ID`
  """)
  return aggregate_df
  

In [0]:
try:
  aggregate_df = aggregate_profit_by_year('enriched_im_t') 
  test_profit_by_year(aggregate_df)
  print("-------Profit by Year-----")
  aggregate_df.show()
  print("year aggregate table test passed!")
except AssertionError as e:
  print("aggregate table test failed:", e)


try:
  aggregate_df = aggregate_profit_by_year_category('enriched_im_t') 
  test_profit_by_year_category(aggregate_df)
  print("-------Profit by Year + Category-----")
  aggregate_df.show()
  print("year + category aggregate table test passed!")
except AssertionError as e:
  print("year + category aggregate table test failed:", e)

-------Profit by Year-----
+----+------------+
Year|Total_Profit|
+----+------------+
2015| 62490.14|
2014| 38662.76|
2016| 64669.14|
2017| 110596.8|
+----+------------+

year aggregate table test passed!
-------Profit by Year + Category-----
+----+----------------+------------+
Year|Product_Category|Total_Profit|
+----+----------------+------------+
2017| Office Supplies| 44273.35|
2015| Office Supplies| 24519.51|
2016| Technology| 23223.84|
2014| Technology| 21493.38|
2015| Furniture| 3027.2|
2014| Furniture| -5331.05|
2017| Technology| 63281.91|
2017| Furniture| 3041.54|
2015| Technology| 34943.43|
2014| Office Supplies| 22500.43|
2016| Furniture| 6889.56|
2016| Office Supplies| 34555.74|
+----+----------------+------------+

year + category aggregate table test passed!

In [0]:
try:
  aggregate_df = aggregate_profit_by_customer('enriched_im_t') 
  def test_profit_by_customer(aggregated_df):
    aggregated_df.show()
  print("-------Profit by Customer-----")
  aggregate_df.show()
  print("Custoemr aggregate table test passed!")
except AssertionError as e:
  print("Customer aggregate table test failed:", e)

-------Profit by Customer-----
+--------+------------+
Customer|Total_Profit|
+--------+------------+
VW-21775| -874.66|
PB-19210| 21.9|
RR-19315| -73.83|
MY-17380| 294.82|
EM-13960| 102.3|
MS-17530| 81.77|
KH-16630| 726.1|
SW-20275| 332.76|
AH-10690| 1239.86|
BD-11500| 1127.49|
PH-18790| 47.89|
JF-15490| -18.05|
JF-15415| 624.41|
IM-15070| 514.2|
PW-19240| 1283.37|
NW-18400| 1002.41|
KF-16285| 1660.41|
OT-18730| -925.56|
KD-16615| 539.72|
JH-15985| -644.48|
+--------+------------+
only showing top 20 rows

Custoemr aggregate table test passed!

In [0]:
try:
  aggregate_df = aggregate_profit_customer_by_year('enriched_im_t') 
  test_profit_by_customer_year(aggregate_df)
  print("-------Profit by Year + customer-----")
  aggregate_df.show()
  print("year + customer aggregate table test passed!")
except AssertionError as e:
  print("year + customer aggregate table test failed:", e)

-------Profit by Year + customer-----
+----+--------+------------+
Year|Customer|Total_Profit|
+----+--------+------------+
2014|ZD-21925| 25.49|
2015|PW-19240| 1370.23|
2017|MF-18250| 287.24|
2015|LO-17170| 34.45|
2017|TB-21250| -31.68|
2015|SU-20665| 76.4|
2014|TZ-21580| 132.3|
2017|ML-17395| 11.1|
2017|JM-15580| -0.6|
2015|BS-11590| 103.1|
2015|RM-19375| 19.97|
2017|DS-13180| -29.56|
2017|HM-14980| 41.23|
2014|BP-11095| 131.71|
2015|JW-15955| -201.65|
2015|SO-20335| -282.31|
2015|BP-11230| -255.74|
2016|GB-14575| 73.22|
2016|TW-21025| 33.01|
2016|TT-21070| 2.69|
+----+--------+------------+
only showing top 20 rows

year + customer aggregate table test passed!